In [ ]:
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
import pandas as pd
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, UpSampling2D
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals import joblib
import imageio as io
from pylab import *
from sklearn.manifold import TSNE
from keras import backend as K
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tensorflow.keras.models import Sequential
from keras.models import load_model

In [ ]:
# all_images = [file for file in os.listdir('/Users/jennysheng/Documents/museum_images/all_image_data/') if file.endswith(".jpg")]
# train_images, test_images = train_test_split(all_images, test_size = 0.05)

# train_images = pd.DataFrame(train_images,columns=['filepath'])
# test_images = pd.DataFrame(test_images,columns=['filepath'])
train_images = pd.read_csv('/Users/jennysheng/Documents/museum_images/train_file.csv')
test_images = pd.read_csv('/Users/jennysheng/Documents/museum_images/test_file.csv')
train_images = list(train_images['filepath'])
test_images = list(test_images['filepath'])

In [ ]:
# helper function to reshape images into tensors
def image_tensors(file_array):
    image_array = []
    for image_path in tqdm(file_array):
        image = cv2.imread('/Users/jennysheng/Documents/museum_images/all_image_data/'+ image_path)
        print(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224,224))
        image_array.append(np.array(image))
    image_array = np.array(image_array)
    image_array = image_array.reshape(image_array.shape[0], 224, 224, 3).astype('float32')/255 
    return np.array(image_array)

test_data = image_tensors(test_images)
train_data = image_tensors(train_images)

In [ ]:
def encoder_decoder():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=(224, 224, 3),padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    model.add(Conv2D(128, kernel_size=(3, 3),strides=1,kernel_regularizer = tf.keras.regularizers.L2(0.001),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu',kernel_regularizer= tf.keras.regularizers.L2(0.001), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu',kernel_regularizer= tf.keras.regularizers.L2(0.001), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,padding='valid'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))
    
    model.add(Conv2D(512, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001),activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(512, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(256, kernel_size=(3, 3), kernel_regularizer = tf.keras.regularizers.L2(0.001), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', kernel_regularizer = tf.keras.regularizers.L2(0.001), padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer = tf.keras.regularizers.L2(0.001), padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, kernel_size=(3, 3), padding='same',activation='sigmoid'))
    return model

In [ ]:
model = encoder_decoder()
model.summary()

In [ ]:
# train using Adam optmizer
early_stopping = EarlyStopping(monitor='val_loss', mode='min',verbose=1,patience=6,min_delta=0.0001) 
checkpoint = ModelCheckpoint('/Users/jennysheng/Documents/museum_images/encoder_model.h5', monitor='val_loss', mode='min', save_best_only=True) 
optimizer = Adam(learning_rate=0.001) 
model = encoder_decoder() 
model.compile(optimizer=optimizer, loss='mse') 
model.fit(train_data, train_data, epochs=20, batch_size=32,validation_data=(test_data,test_data),callbacks=[early_stopping,checkpoint]) 